In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import glob

import numpy as np
import tensorflow as tf

%matplotlib inline

In [ ]:
# data import
data_dir = "cars_data/"
train_x = np.array(pickle.load(open(data_dir + "train_feature.pkl", "br")), dtype="float32")
train_y = np.array(pickle.load(open(data_dir + "train_label.pkl", "br")), dtype="float32")
valid_x = np.array(pickle.load(open(data_dir + "valid_feature.pkl", "br")), dtype="float32")
valid_y = np.array(pickle.load(open(data_dir + "valid_label.pkl", "br")), dtype="float32")
test_x = np.array(pickle.load(open(data_dir + "test_feature.pkl", "br")), dtype="float32")
test_y = np.array(pickle.load(open(data_dir + "test_label.pkl", "br")), dtype="float32")

In [ ]:
train_x[:10]

In [ ]:
train_y[:10]

In [ ]:
len(train_x)

In [ ]:
len(valid_x)

In [ ]:
len(test_x)

In [ ]:
# define placeholder tensors which work as "entrances" of the data
x1 = tf.placeholder(dtype=tf.float32, shape=[None, None])
y = tf.placeholder(dtype=tf.int32, shape=[None,])

In [ ]:
# define parameters for the 1st layer
W1 = tf.Variable(tf.random_normal([6, 20], stddev=0.05), dtype=tf.float32)
b1 = tf.Variable(tf.zeros((1, 20)), dtype=tf.float32)
# tensor calculation for the 1st layer
a2 = tf.matmul(x1, W1) + b1
x2 = tf.sigmoid(a2)

# define parameters for the 2nd layer
W2 = tf.Variable(tf.random_normal([20, 8], stddev=0.05), dtype=tf.float32)
b2 = tf.Variable(tf.zeros((1, 8)), dtype=tf.float32)
# tensor calculation for the 2nd layer
a3 = tf.matmul(x2, W2) + b2
x3 = tf.sigmoid(a3)

# the 3rd layer
W3 = tf.Variable(tf.random_normal([8, 4], stddev=0.05), dtype=tf.float32)
b3 = tf.Variable(tf.zeros((1, 4)), dtype=tf.float32)
logits = tf.matmul(x3, W3) + b3

In [ ]:
# the average cost for records in the batch:
# tf calculates it with the logits and y labels
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
cost = tf.reduce_mean(cross_entropy)

In [ ]:
# define the train op
alpha = 1.
train_op = tf.train.GradientDescentOptimizer(alpha).minimize(cost)

In [ ]:
# prepare a dir to store model parameters
if not os.path.isdir("cars_models"):
    ! mkdir cars_models
saver = tf.train.Saver(max_to_keep=None)

In [ ]:
def evaluation(sess, d_x, d_y):
    raw_results = sess.run(logits, feed_dict={x1: d_x})
    results = np.argmax(raw_results, axis=1)
    match_count = np.sum(d_y == results)
    accuracy = float(match_count) / float(d_y.size)
    
    return accuracy

In [ ]:
# training

total_epochs = 2000
cost_history = list()
valid_history = list()
best_iter = 0
test_accuracy = 0.

# clear the past model dir
if len(glob.glob("cars_models/model.*")) > 0:
    ! rm cars_models/model.*
    
# setup a session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for itr in range(1, total_epochs + 1):
        fd = {x1: train_x, y: train_y}
        cost_val, t = sess.run((cost, train_op), feed_dict=fd)
        
        # store and display the status once in ten iterations
        if itr % 10 == 0:
            cost_history.append((itr, cost_val))
            print("itr: {}, cost: {:.3f}".format(itr, cost_val))
            
        # validation
        if itr % 50 == 0:
            acc = evaluation(sess, valid_x, valid_y)
            valid_history.append((itr, acc))
            print("validation accuracy: {:.3f}".format(acc))
            print("")
            saver.save(sess, "cars_models/model.{}".format(itr))
            
        itr += 1
        
    # test
    sorted_valid_history = sorted(valid_history, key=lambda x: x[1], reverse=True)
    best_itr = sorted_valid_history[0][0]
    best_acc = sorted_valid_history[0][1]
    print("best validation performance was {:.3f} at iter {}".format(best_acc, best_itr))
    
    # restore the model and evaluate it with the test data
    saver.restore(sess, "cars_models/model.{}".format(best_itr))
    test_accuracy = evaluation(sess, test_x, test_y)
    print("test accuracy: {:.3f}".format(test_accuracy))
        

In [ ]:
# visualize the result

plt.figure(figsize=(10, 5))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)

plt.subplots_adjust(hspace=.4)

train_x_val = [t[0] for t in cost_history]
train_y_val = [t[1] for t in cost_history]
valid_x_val = [t[0] for t in valid_history]
valid_y_val = [t[1] for t in valid_history]

ax1.plot(train_x_val, train_y_val, color="green")
ax2.plot(valid_x_val, valid_y_val)
ax2.hlines(test_accuracy, xmin=train_x_val[0], xmax=train_x_val[-1], color="red", linestyles="--")
ax2.vlines(best_itr, ymin=np.min(valid_y_val), ymax=np.max(valid_y_val), color="blue", linestyles=":")

plt.title("learning curve for the cars training")
ax1.set_title("cost")
ax2.set_title("accuracy")
ax2.set_xlabel("# of iterations");